In [ ]:
from chess import Board, pgn
from auxiliary_func import board_to_matrix
import torch
from model import ChessModel
import pickle
import numpy as np

# Predictions

1. Prepare the Input Data
Convert the board state to a format suitable for the model:

In [ ]:
def prepare_input(board: Board):
    matrix = board_to_matrix(board)
    X_tensor = torch.tensor(matrix, dtype=torch.float32).unsqueeze(0)
    return X_tensor

2. Load the Model & mapping and Move to GPU if Available

In [ ]:
# Load the mapping

with open("../../models/heavy_move_to_int", "rb") as file:
    move_to_int = pickle.load(file)

# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

# Load the model
model = ChessModel(num_classes=len(move_to_int))
model.load_state_dict(torch.load("../../models/TORCH_100EPOCHS.pth"))
model.to(device)
model.eval()  # Set the model to evaluation mode (it may be reductant)

int_to_move = {v: k for k, v in move_to_int.items()}
# Function to make predictions
def predict_move(board: Board):
    X_tensor = prepare_input(board).to(device)
    
    with torch.no_grad():
        logits = model(X_tensor)
    
    logits = logits.squeeze(0)  # Remove batch dimension
    
    probabilities = torch.softmax(logits, dim=0).cpu().numpy()  # Convert to probabilities
    legal_moves = list(board.legal_moves)
    legal_moves_uci = [move.uci() for move in legal_moves]
    sorted_indices = np.argsort(probabilities)[::-1]
    for move_index in sorted_indices:
        move = int_to_move[move_index]
        if move in legal_moves_uci:
            return move
    
    return None

3. Use the ```predict_move``` function to get the best move and its probabilities for a given board state:

In [ ]:
# Initialize a chess board
board = Board()

In [ ]:
board

In [ ]:
board.push_uci("e2e4")

In [ ]:
board

In [ ]:
# Predict and make a move
best_move = predict_move(board)
board.push_uci(best_move)
board

In [ ]:
print(str(pgn.Game.from_board(board)))